# Migrationshintergrund

- Line/Barchart
- Gibt es andere Beeinträchtigungen?

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

## Import Data

In [161]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Das erste Blatt entfernen
sheets.pop(next(iter(sheets)))  # Entfernt das erste Blatt basierend auf der Reihenfolge

# Dictionary zur Speicherung der Werte
gewichteter_anteil = {}  # Gewichtete relative Quote basierend auf der tatsächlichen Gesamtzahl

# Durch alle verbleibenden Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()
    
    # Extrahiere die Zeilen mit "Migrationshintergrund (%)"
    migration_row = df[df["Variable"].str.contains("Migrationshintergrund", na=False) & 
                       df["Category"].str.contains("\(\%\)", na=False)]
    
    # Extrahiere die Zeilen mit "Anzahl Studierende"
    total_students_row = df[df["Variable"].str.contains("Migrationshintergrund", na=False) & 
                            df["Category"].str.contains("Anzahl", na=False)]
    
    if not migration_row.empty and not total_students_row.empty:
        # Gesamtanzahl der Studierenden summieren
        total_students = total_students_row.iloc[:, 2:].sum(axis=1).values[0]

        # Mittelwert des Anteils über alle Studiengänge berechnen
        avg_migration_rate = migration_row.iloc[:, 2:].mean(axis=1).values[0]

        # Gewichteter relativer Anteil berechnet als: (absolute Anzahl / Gesamtzahl der Studierenden) * 100
        gewichteter_anteil[jahr] = (avg_migration_rate / 100) * 100

# Daten als DataFrame formatieren
df_migration = pd.DataFrame({
    "Jahr": list(gewichteter_anteil.keys()),
    "Gewichteter Relativer Anteil (%)": list(gewichteter_anteil.values())
})

df_migration = df_migration.sort_values(by="Jahr")

# Plot mit Plotly erstellen
fig = px.line(
    df_migration, 
    x="Jahr", 
    y="Gewichteter Relativer Anteil (%)", 
    markers=True, 
    title="Relativer Anteil Studierender mit Migrationshintergrund",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Layout-Anpassungen
fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 40]),  # Y-Achsenbereich festlegen
    annotations=[
        dict(
            text="Datenquelle: Universität 2024",  # Individueller Untertitel
            xanchor="left",  # Linksbündig
            x=-0.045,  # x-Position
            y=1.14,  # y-Position
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=14, color="grey", family="Arial")  # Schriftgröße, Farbe und Schriftart
        )
    ],
    xaxis=dict(range=[2013, 2025]) 
)

fig.show()

<>:22: SyntaxWarning:

invalid escape sequence '\('

<>:22: SyntaxWarning:

invalid escape sequence '\('

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_27882/4195163379.py:22: SyntaxWarning:

invalid escape sequence '\('



In [152]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [205]:
import pandas as pd

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Neuer Container für die angepassten DataFrames
updated_sheets = {}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen

    # 🔹 Extrahiere die richtige Zeile für "Migrationshintergrund (%)"
    migration_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Migrationshintergrund (%)")
    ]

    # 🔹 Extrahiere die Zeile für "Anzahl Studierende"
    total_students_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Anzahl")
    ]


    # 🔹 Gemeinsame Studiengangs-Spalten bestimmen (alle außer "Variable" und "Category")
    common_columns = migration_row.columns.intersection(total_students_row.columns)[2:]  # Ab Spalte 2
 

    # 🔹 Konvertiere Werte zu Float und ersetze NaN durch 0
    migration_values = migration_row[common_columns].astype(float).fillna(0).squeeze()
    total_students_values = total_students_row[common_columns].astype(float).fillna(0).squeeze()
    
    # 🔹 Berechne absolute Anzahl Studierender mit Migrationshintergrund pro Studiengang
    abs_migration_students_per_studiengang = (migration_values / 100) * total_students_values

    # 🔹 Neue Zeile für absolute Anzahl erstellen
    new_row = pd.DataFrame([["Migrationshintergrund", "Absolute Anzahl"] + abs_migration_students_per_studiengang.tolist()], columns=df.columns)

    # 🔹 DataFrame aktualisieren
    df = pd.concat([df, new_row], ignore_index=True)

    # 🔹 Modifizierten DataFrame speichern
    updated_sheets[jahr] = df

# ✅ Fertige, aktualisierte DataFrames liegen jetzt in `updated_sheets`

In [204]:
updated_sheets["2024"]

,Variable,Category,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Agrar- und Ressourcenökonomie,Master Agrobiotechnologie,Master Ernährungswissenschaften,...,Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,Master Insect Biotechnology and Bioresources,Master Nachhaltige Ernährungswirtschaft,Master Nutzpflanzenwissenschaften,Master Nutztierwissenschaften*,Master Oenologie/Weinwirtschaft*,Master Ökotrophologie,Master Sustainable Transition,Master Transition Management,Master Umweltwissenschaften
0,Geschlecht,Männlich (%),57,8.00,25.00,11.0,21.00,38.00,22.00,3.00,...,NaN,40.0,9.00,NaN,NaN,NaN,14.00,20.0,58.00,35.00
1,Geschlecht,Weiblich (%),43,92.00,67.00,86.0,77.00,63.00,78.00,97.00,...,NaN,60.0,91.00,NaN,NaN,NaN,86.00,80.0,42.00,65.00
2,Geschlecht,Gesamt (%),100,100.00,100.00,100.0,100.00,100.00,100.00,100.00,...,NaN,100.0,100.00,100.0,NaN,NaN,100.00,100.0,100.00,100.00
3,Geschlecht,Anzahl,23,63.00,12.00,44.0,52.00,8.00,27.00,33.00,...,NaN,10.0,11.00,3.0,NaN,NaN,22.00,5.0,19.00,17.00
4,Alter,unter 18 Jahre (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Studienbeeinträchtigungen,fehlende Berufsperspektiven (%),10,9.00,29.00,14.0,18.00,10.00,33.00,21.00,...,NaN,31.0,NaN,NaN,NaN,NaN,13.00,NaN,24.00,18.00
66,Rücklaufquote,Anzahl Studierende,293,493.00,69.00,429.0,116.00,53.00,139.00,338.00,...,13.0,81.0,51.00,65.0,41.0,32.0,152.00,59.0,192.00,13.00
67,Rücklaufquote,Anzahl Teilnahmen an Studierendenbefragung,34,104.00,18.00,59.0,63.00,14.00,36.00,38.00,...,3.0,17.0,14.00,6.0,4.0,3.0,28.00,7.0,22.00,3.00
68,Rücklaufquote,Anteil Teilnehmer (%),12,21.00,26.00,14.0,54.00,26.00,26.00,11.00,...,23.0,21.0,27.00,9.0,10.0,9.0,18.00,12.0,11.00,23.00


In [206]:
import pandas as pd

# Dictionary zur Speicherung der gewichteten Anteile pro Jahr
gewichteter_anteil = {}

# Durch alle Jahre iterieren
for jahr, df in updated_sheets.items():
    # Extrahiere die Zeile für "Migrationshintergrund" - Absolute Anzahl
    abs_migration_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Absolute Anzahl")
    ]

    # Extrahiere die Zeile für "Migrationshintergrund" - Anzahl
    total_students_row = df[
        (df["Variable"] == "Migrationshintergrund") & 
        (df["Category"] == "Anzahl")
    ]

    # Falls keine Daten vorhanden sind, überspringen
    if abs_migration_row.empty or total_students_row.empty:
        print(f"Warnung: Keine Migrationshintergrund-Daten für {jahr} gefunden!")
        continue  

    # Relevante Spalten (Studiengänge) extrahieren
    common_columns = abs_migration_row.columns.intersection(total_students_row.columns)[2:]

    # Gesamtanzahl der Studierenden mit Migrationshintergrund und aller Studierenden berechnen
    abs_migration_total = abs_migration_row[common_columns].sum().sum()
    total_students = total_students_row[common_columns].sum().sum()

    # Gewichteten relativen Anteil berechnen
    if total_students > 0:
        gewichteter_anteil[jahr] = (abs_migration_total / total_students) * 100
    else:
        gewichteter_anteil[jahr] = 0  # Falls keine Studierenden erfasst wurden

# Daten als DataFrame formatieren
migrations_df = pd.DataFrame({
    "Jahr": list(gewichteter_anteil.keys()),
    "Gewichteter Relativer Anteil (%)": list(gewichteter_anteil.values())
})

migrations_df = migrations_df.sort_values(by="Jahr")

In [207]:
migrations_df

,Jahr,Gewichteter Relativer Anteil (%)
0,2013,0.000000
1,2014,16.232432
2,2015,13.033573
3,2016,15.093156
4,2017,16.355670
5,2018,17.921002
6,2019,16.577444
7,2020,20.585911
8,2021,20.985915
9,2022,26.267504


In [208]:
import plotly.express as px

fig = px.line(
    migrations_df, 
    x="Jahr", 
    y="Gewichteter Relativer Anteil (%)", 
    markers=True, 
    title="Relativer Anteil Studierender mit Migrationshintergrund",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 40]),  # Y-Achsenbereich festlegen
    xaxis=dict(range=[2013, 2025]) 
)

fig.show()